In [1]:
# Code to analyze emergency closure data

"""
Here are codes used by state inspectors when they determine
a restaurant should be closed temporarily. This is taken from
http://www.myfloridalicense.com/DBPR/hotels-restaurants/inspections/inspection-dispositions/


Facility Temporarily Closed:
Operations ordered stopped until violations are corrected
The inspector recommended closing the facility immediately
after finding conditions that may endanger the health and
safety of the public.

Dispositions included in this result are:

Emergency order recommended – Conditions have been found
that endanger the health and safety of the public requiring
immediate closure of the establishment.

Administrative determination recommended – The establishment
is operating without a license and action is being taken
to ensure proper licensing is completed.

Emergency Order Callback Not Complied – Corrections to violations
that resulted in an emergency order were not completed
at the time of inspection. Violations may not be noted
again on these inspection reports.
"""

import csv
import pandas as pd
import numpy as np
import sqlite3

# Set up connection to database.

imacpath = "/Users/rayd/workspace/flinsp/datafiles/"
airpath = "/Users/Doug/workspace/flinsp/datafiles/"
dbfile = "rinspect18.sqlite"
dbpath = airpath + dbfile
conn = sqlite3.connect(dbpath)

# Make a list of visitid numbers for restaurants shut down
# by inspectors.

conn.row_factory = lambda cursor, row: row[0]
c = conn.cursor()

# on initial inspection
vids = c.execute("SELECT visitid FROM fdinsp WHERE inspdispos = 'Emergency order recommended'").fetchall()

# remained shut on subsequent inspection
vids2 = c.execute("SELECT visitid FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Not Complied'").fetchall()

# Make pandas dataframe for both sets

conn = sqlite3.connect(dbpath)

df = pd.read_sql_query("SELECT * FROM fdinsp WHERE inspdispos = 'Emergency order recommended';", conn)
df2 = pd.read_sql_query("SELECT * FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Not Complied';", conn)

# Test to see if our dataset is complete

count1 = df.shape
count2 = df2.shape
print("There were " + str(count1[0]) + " restaurants closed on first inspection.")
print("\nThere were " + str(count2[0]) + " remained closed on subsequent inspection.")

# Contains 'Emergency' but means reopened
df_test1 = pd.read_sql_query(
    "SELECT * FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Complied';", conn
    )
count3 = df_test1.shape
print("\nThere were " + str(count3[0]) + " cleared and opened by subsequent inspection.")

# Contains 'Emergency' but also means reopened
df_test2 = pd.read_sql_query(
    "SELECT * FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Time Extension';", conn
    )
count4 = df_test2.shape
print("\nThere were " + str(count4[0]) + " reopened but will need another inspection.")

print("\n" +
    str(count1[0]) + " + " +
    str(count2[0]) + " + " +
    str(count3[0]) + " + " +
    str(count4[0]) + " + " +
    " = " + str(count1[0] + count2[0] + count3[0] + count4[0])
     )

# Contains something like 'Emergency' but are there some where spelling or capitalization shifts?
df_test3 = pd.read_sql_query("SELECT * FROM fdinsp WHERE inspdispos LIKE '%mergency%';", conn)
count5 = df_test3.shape
print("\nThere were " + str(count5[0]) + " that had some word like 'emergency'.")

print("\nSo it looks like we got them all.")


There were 1238 restaurants closed on first inspection.

There were 547 remained closed on subsequent inspection.

There were 810 cleared and opened by subsequent inspection.

There were 365 reopened but will need another inspection.

1238 + 547 + 810 + 365 +  = 2960

There were 2960 that had some word like 'emergency'.

So it looks like we got them all.


In [ ]:
# Create list of dictionaries with detailed inspection reports
# that led to closures

def dict_factory(cursor, row):
    dvio = {}
    for idx, col in enumerate(cursor.description):
        dvio[col[0]] = row[idx]
    return dvio

lvio = []
lvio2 = []

for vid in vids:
    con = sqlite3.connect(dbpath)
    con.row_factory = dict_factory
    cur = con.cursor()
    cur.execute(f"SELECT * FROM violations WHERE visitid = {vid}")
    lvio.extend(cur.fetchall())
    con.close()

for vid2 in vids2:
    con = sqlite3.connect(dbpath)
    con.row_factory = dict_factory
    cur = con.cursor()
    cur.execute(f"SELECT * FROM violations WHERE visitid = {vid2}")
    lvio2.extend(cur.fetchall())
    con.close()


In [ ]:
# Write csv files for violation details

keys = lvio[0].keys

with open('closurevios.csv', 'w', newline='') as output_file:
    fc = csv.DictWriter(output_file,
                        fieldnames=lvio[0].keys()
                       )

    fc.writeheader()
    fc.writerows(lvio)

keys = lvio2[0].keys

with open('closurevios2.csv', 'w', newline='') as output_file:
    fc = csv.DictWriter(output_file,
                        fieldnames=lvio2[0].keys()
                       )

    fc.writeheader()
    fc.writerows(lvio2)


In [ ]:
# Make dataframes with violation details

df3 = pd.DataFrame(lvio) # Closed on initial inspection
df4 = pd.DataFrame(lvio2) # Remained closed after follow-up

# What was the most common violation in a closure inspection?

df3g = df3.groupby('violation').count().sort_values(by=['visitid'], axis=0, ascending=False)

# What were the most common violations in a closure inspection?

df3.groupby('violation').count().sort_values(
    by=['visitid'], axis=0, ascending=False
    ).head(10)


In [ ]:
# Violation codes and descriptions
a = '35A-01-4' # Intermediate: Service animals
b = '35A-02-5' # High priority: Live, small flying insects in food service area
c = '35A-03-4' # Basic: Dead roaches on premesis
'35A-04-4' # High priority: Rodent activity present as evidenced by droppings
'35A-05-4' # High priority: Live roaches found
'35A-06-4' # Basic: Accumulation of dead or trapped pests
'35A-07-4' # High priority: Small flying insects in bar, kitchen, dumster, prep area
'35A-09-4' # High priority: Presence of insects, rodents or other pests
'35A-18-4' # High priority: Rodent rub marks present
'35A-20-4' # Basoc: Dead rodent present
'35A-21-4' # High priority: Rodent burrow or nesting materials present
'35A-23-4' # High priority: Rodent droppings present
'03A-02-4' # High priority: Potentially hazardous  cold food held at greater than 41 degrees


In [ ]:
df3g2 = df3.groupby('violation')
df3g2

In [ ]:
# Are there any closures that don't involve 35A-0*'?
# *** THIS ISNT WORKING YET ***

value = '35A-02-5'

print(df.groupby('visitid').filter(lambda x: all(value != i for i in x['violation'])))

# dfa = df3[df3['violation'].str.contains("35A-0")]
# dfb = df3[~df3['violation'].str.contains("35A-0")]


In [ ]:
# Which counties had the most closures?
# Calculated as closers per licensed restaurant

# Count closures per county
dfc = df.groupby('county').count()
dfc = dfc.licnum.reset_index()
dfc = dfc.rename(columns={'county' : 'county', 'licnum' : 'closures'})
dfc = dfc.set_index('county')

# Which counties are included in closures
co_inc = list(df.groupby(['county']).groups.keys())

#List of all Florida counties
with open('counties.txt', 'r') as f:
    fl_counties = [line.rstrip('\n') for line in f]

def diff(co_inc, fl_counties):
    co_dif = [i for i in co_inc + fl_counties if i not in co_inc]
    return co_dif

missing_counties = diff(co_inc, fl_counties)

print("\nDid any counties not have closure orders in FY2018-19?")
print("\nThese are not included: " + str(', '.join(missing_counties)))
print("\nBut Miami-Dade listed simply as Dade in our data frame.")

missing = list(missing_counties)
missing.remove('Miami-Dade')


In [ ]:
# Read in csv of licensed restaurants per county
df_cntylic = pd.read_csv('countycount.csv')
df_cntylic = df_cntylic.drop(['Unnamed: 0'], axis=1)
df_cntylic = df_cntylic[~df_cntylic['co_name'].isin(missing)] # drop missing counties
df_cntylic= df_cntylic.rename(columns={"lic_count": "licenses", "co_name": "county"})
df_cntylic = df_cntylic.set_index('county')


In [ ]:
# Closures per license
dfc = df_cntylic.join(dfc)


In [ ]:
dfc['ratio'] = dfc.closures / dfc.licenses
dfc['percent'] = dfc.ratio * 100
dfc = dfc.sort_values(by=['ratio'])
most_closed = dfc.sort_values(by=['ratio'], ascending=False).head(10)
least_closed  = dfc.sort_values(by=['ratio'], ascending=True).head(10)

In [ ]:
most_closed.head(20)

In [ ]:
least_closed.head(20)